# Simularium Conversion Tutorial : CytoSim Data

In [1]:
from IPython.display import Image
import numpy as np
from simulariumio import CytosimConverter, CytosimData, CytosimObjectInfo, CytosimAgentInfo

This notebook provides example python code for converting your own simulation trajectories into the format consumed by the Simularium Viewer. It creates a .simularium JSON file which you can drag and drop onto the viewer like this:

![title](img/drag_drop.gif)

***
## Prepare your spatial data

The Simularium `CytosimConverter` consumes spatiotemporal data from CytoSim. 

To see how to generate Cytosim output .txt files, check Cytosim documentation here: https://gitlab.com/f.nedelec/cytosim-stable/-/blob/master/doc/sim/report.md
* for Fibers, use the command `./report fiber:points > fiber_points.txt`, which will create `fiber_points.txt`
* for Solids, use the command `./report solid > solids.txt`, which will create `solids.txt`
* for Singles, use the command `./report single:position > singles.txt`, which will create `singles.txt`
* for Couples, use the command `./report couple:state > couples.txt`, which will create `couples.txt`
    * in some versions of Cytosim, state is not a reporting option. In this case you can use `./report couple:[name of your couple] > couples_[name of your couple].txt` and provide a filepath for each type of couple in your data. If this is necessary, you should also check the position XYZ columns in your `couples.txt` file and override **position_indices** if they aren't at \[2, 3, 4\]

The converter requires a `CytosimData` object as parameter.

`CytosimData` contains the following:
* **box_size** : *np.ndarray (shape = \[3\])*
    * A numpy ndarray containing the XYZ dimensions of the simulation bounding volume
        * *for now we only support rectangular bounds, but will be adding support for spheres soon*
* **object_info** : *Dict\[str, `CytosimObjectInfo`\]*
    * A dict mapping Cytosim object type (either "fibers", "solids", "singles", or "couples") to info for reading the Cytosim data for agents of that object type
* **draw_fiber_points** : *bool (optional)*
    * (only used for fibers)
    * in addition to drawing a line for each fiber, also draw spheres at every other point along it?
    * Default: False
* **scale_factor** : *float (optional)*
    * A multiplier for the Cytosim scene, use if visualization is too large or small
    * Default: 1.0
* **plots** : *List\[Dict\[str, Any\]\] (optional)*
    * An object containing plot data already in Simularium format
    
`CytosimObjectInfo` contains the following:
* **filepath** : *str*
    * A string path to fiber_points.txt
* **agents** : *Dict\[int, `CytosimAgentInfo`\] (optional)*
    * A dict mapping the type index from Cytosim data to display names and radii for each type of agent of this type of Cytosim object
* **position_indices** : *List\[int\] (optional)*
    * the columns in Cytosim's reports are not always consistent, use this to override them if your output file has different column indices for position XYZ
    * Default: \[2, 3, 4\]
    
`CytosimAgentInfo` contains the following:
* **name** : *str*
    * A string display name for this type of agent
    * Default: "[Cytosim object type][agent type index from Cytosim data]" e.g. "fiber1", "solid0"
* **radius** : *float (optional)*
    * A float radius for rendering this agent.
    * For fibers, this is the thickness of the line
    * For default agents, this is the radius of the sphere
    * Default : 1.0
  

In [2]:
box_size = 2.

example_data = CytosimData(
    box_size=np.array([box_size, box_size, box_size]),
    object_info={
        "fibers" : CytosimObjectInfo(
            filepath="../simulariumio/tests/data/cytosim/aster_pull3D_couples_actin_solid/fiber_points.txt",
            agents={
                "1" : CytosimAgentInfo(
                    name="microtubule"
                ),
                "2" : CytosimAgentInfo(
                    name="actin"
                )
            }
        ),
        "solids" : CytosimObjectInfo(
            filepath="../simulariumio/tests/data/cytosim/aster_pull3D_couples_actin_solid/solids.txt",
            agents={
                "1" : CytosimAgentInfo(
                    name="aster",
                    radius=0.1
                ),
                "2" : CytosimAgentInfo(
                    name="vesicle",
                    radius=0.1
                )
            }
        ),
        "singles" : CytosimObjectInfo(
            filepath="../simulariumio/tests/data/cytosim/aster_pull3D_couples_actin_solid/singles.txt",
            agents={
                "1" : CytosimAgentInfo(
                    name="dynein",
                    radius=0.01
                ),
                "2" : CytosimAgentInfo(
                    name="kinesin",
                    radius=0.01
                )
            }
        ),
        "couples" : CytosimObjectInfo(
            filepath="../simulariumio/tests/data/cytosim/aster_pull3D_couples_actin_solid/couples.txt",
            agents={
                "1" : CytosimAgentInfo(
                    name="motor complex",
                    radius=0.02
                )
            },
            position_indices=[3, 4, 5]
        )
    },
    scale_factor=100.0
)

## Convert and save as .simularium JSON file

Once your data is shaped like in the `example_data` object, you can use the converter to generate the file at the given path:

In [3]:
CytosimConverter(example_data).write_JSON("cytosim_example")

## Visualize in the Simularium viewer

In a supported web-browser (Firefox or Chrome), navigate to https://staging.simularium.allencell.org/ and import your file into the view.